In [1]:
!pip install sentence-transformers 

     |████████████████████████████████| 78 kB 682 kB/s            
  Preparing metadata (setup.py) ... done
  Created wheel for sentence-transformers: filename=sentence_transformers-2.1.0-py3-none-any.whl size=121000 sha256=47f9214d658797f2878e2c27b04dd151bbe4863e525a3a1c4713e0e4ac283ada
  Stored in directory: /root/.cache/pip/wheels/90/f0/bb/ed1add84da70092ea526466eadc2bfb197c4bcb8d4fa5f7bad
Successfully built sentence-transformers


In [2]:
!pip install keybert

  Preparing metadata (setup.py) ... done
     |████████████████████████████████| 215 kB 608 kB/s            
     |████████████████████████████████| 51 kB 3.3 MB/s            
  Created wheel for keybert: filename=keybert-0.5.0-py3-none-any.whl size=20491 sha256=34d09646f5555267eef38313b6423609acd0044e6a07c4c7821ef638f4181c82
  Stored in directory: /root/.cache/pip/wheels/99/1f/3f/590d2997adbb2d0e1f82e8ee05d42d6910e92c3ed283015ff8
Successfully built keybert


In [3]:
import numpy as np
import pandas as pd   
import sklearn 
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import manhattan_distances
from sklearn.metrics.pairwise import euclidean_distances
from sklearn import metrics  
from scipy.spatial import distance
from sentence_transformers import SentenceTransformer 
from keybert import KeyBERT
kw_model = KeyBERT()
model = SentenceTransformer('distilbert-base-nli-mean-tokens')

Downloading:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/10.2k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/612 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/116 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/39.3k [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/450 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [5]:
dataframe=pd.read_csv("../input/testdata/data.csv")
dataframe.shape

(24, 4)

In [ ]:
# standard_answer=dataframe['Questioner Answer'][6]
# student_answer=dataframe['''Student's Answer'''][6] 

In [6]:
class SimilarityMetric:
    def __init__(self,student_answer,standard_answer,st_ans,sn_ans,model=model) -> None:
        self.student_answer=student_answer
        self.standard_answer=standard_answer 
        self.st_ans=st_ans
        self.sn_ans=sn_ans 
        self.embded_student_answer = model.encode(student_answer)
        self.embded_standard_answer= model.encode(standard_answer)

    def euclidian_dist(self): 
        dist=euclidean_distances(self.embded_standard_answer,self.embded_student_answer) 
        result=0.0
        for d in dist: 
            result=min(d)
        return result/dist.shape[0]
    
    def manhatten_dist(self):
        dist=manhattan_distances( self.embded_standard_answer,self.embded_student_answer) 
        result=0.0
        for d in dist: 
            result=min(d)
        return result/dist.shape[0]
#         return manhattan_distances( self.embded_standard_answer,self.embded_student_answer) 

    def cosine_similarity(self): 
        distances = cosine_similarity( self.embded_standard_answer,self.embded_student_answer)
        result=0.0
        for d in distances: 
            result=max(d)
        return result/distances.shape[0] 
    
    def Jaccard_Similarity(self): 

        words_doc1 = set(self.st_ans.lower().split()) 
        words_doc2 = set(self.sn_ans.lower().split())

        intersection = words_doc1.intersection(words_doc2)

        union = words_doc1.union(words_doc2)

        return float(len(intersection)) / len(union)



In [7]:
cosine_similar=[]
euclid_dist=[]
manhatten_d=[]
jaccard_Similarity=[]
stnd_ans=[]
stud_ans=[]
_id=[]
actual_sentiments=[]
for i in range(dataframe.shape[0]):
    standard_answer=dataframe['sentence1'][i]
    student_answer=dataframe['sentence2'][i] 
    keywords_stnd = kw_model.extract_keywords([standard_answer], keyphrase_ngram_range=(1, 3), stop_words='english', use_mmr=True, diversity=0.3) 
    keywords_stud = kw_model.extract_keywords([student_answer], keyphrase_ngram_range=(1, 3), stop_words='english', use_mmr=True, diversity=0.3)
    candidates_standard=[]
    candidates_student=[]

    for key,match in keywords_stnd[0]: 
      candidates_standard.append(key)
    for key,match in keywords_stud[0]:
      candidates_student.append(key) 

    sim_obj=SimilarityMetric(candidates_standard,candidates_student,standard_answer,student_answer)
    
    cosine_similar.append(sim_obj.cosine_similarity())
    euclid_dist.append(sim_obj.euclidian_dist())
    manhatten_d.append(sim_obj.manhatten_dist())
    jaccard_Similarity.append(sim_obj.Jaccard_Similarity())
    stnd_ans.append(standard_answer)
    stud_ans.append(student_answer)
    _id.append(i)
    actual_sentiments.append(dataframe['label'][i])
dict_sample = {'Id':_id,'Standard_Answer': stnd_ans, 'Student_Answer': stud_ans,'Actual Sentiment':actual_sentiments,'Cosine_Similarity': cosine_similar,'Euclidian_Distance':euclid_dist,'Manhatten_Distance':manhatten_d,'JaccardSimilarity':jaccard_Similarity,} 
     
df = pd.DataFrame(dict_sample)
   
df.to_csv('KeywordTest.csv')
    

/opt/conda/lib/python3.7/site-packages/keybert/_model.py:130: UserWarning: Although extracting keywords for multiple documents is faster than iterating over single documents, it requires significantly more memory to hold all word embeddings. Use this at your own discretion!
  warnings.warn("Although extracting keywords for multiple documents is faster "
1it [00:00, 399.84it/s]
/opt/conda/lib/python3.7/site-packages/keybert/_model.py:130: UserWarning: Although extracting keywords for multiple documents is faster than iterating over single documents, it requires significantly more memory to hold all word embeddings. Use this at your own discretion!
  warnings.warn("Although extracting keywords for multiple documents is faster "
1it [00:00, 311.13it/s]


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

/opt/conda/lib/python3.7/site-packages/keybert/_model.py:130: UserWarning: Although extracting keywords for multiple documents is faster than iterating over single documents, it requires significantly more memory to hold all word embeddings. Use this at your own discretion!
  warnings.warn("Although extracting keywords for multiple documents is faster "
1it [00:00, 550.07it/s]
/opt/conda/lib/python3.7/site-packages/keybert/_model.py:130: UserWarning: Although extracting keywords for multiple documents is faster than iterating over single documents, it requires significantly more memory to hold all word embeddings. Use this at your own discretion!
  warnings.warn("Although extracting keywords for multiple documents is faster "
1it [00:00, 589.25it/s]


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

/opt/conda/lib/python3.7/site-packages/keybert/_model.py:130: UserWarning: Although extracting keywords for multiple documents is faster than iterating over single documents, it requires significantly more memory to hold all word embeddings. Use this at your own discretion!
  warnings.warn("Although extracting keywords for multiple documents is faster "
1it [00:00, 477.71it/s]
/opt/conda/lib/python3.7/site-packages/keybert/_model.py:130: UserWarning: Although extracting keywords for multiple documents is faster than iterating over single documents, it requires significantly more memory to hold all word embeddings. Use this at your own discretion!
  warnings.warn("Although extracting keywords for multiple documents is faster "
1it [00:00, 449.26it/s]


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
# print(sim_obj.cosine_similarity())
# print(sim_obj.euclidian_dist())
# print(sim_obj.manhatten_dist())
# print(sim_obj.Jaccard_Similarity()) 

In [ ]:
# Jaccard_Similarity(standard_answer,student_answer)

In [ ]:
# def df_plot(result,stndrd_keyword=candidates_standard,student_keyword=candidates_student):
#     return pd.DataFrame(result,columns=student_keyword,index=stndrd_keyword)

In [ ]:
# df_plot(sim_obj.cosine_similarity())

In [ ]:
# df_plot(sim_obj.jaccard_distance())

In [ ]:
# df_plot(sim_obj.euclidian_dist())

In [ ]:
# df_plot(sim_obj.manhatten_dist())